In [5]:
import numpy as np
import pandas as pd
import matplotlib, re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer 
from collections import Counter
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")

## Import and clean the data

In [ ]:
tweets = pd.read_csv("vaccination_all_tweets.csv")
tweets = tweets.dropna(axis=0) 
stop = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
        'ourselves', 'you', "you're", "you've", "you'll", 
        "you'd", 'your', 'yours', 'yourself', 'yourselves',
        'he', 'him', 'his', 'himself', 'she', "she's", 'her',
        'hers', 'herself', 'it', "it's", 'its', 'itself', 'they',
        'them', 'their', 'theirs', 'themselves', 'what', 'which', 
        'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
        'a', 'an', 'the', 'and', 'if', 'or', 'because', 'as', 
        'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
        'between', 'into', 'through', 'during', 'before', 
        'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
        'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
        'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 
        'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 
        'such', 'only', 'own', 'same', 'so', 'than', 
        'too', 'very', 's', 't', 'just',  'now', 'd', 'll', 'm', 'o', 're', 've', 'y']

def remove_stop_words(text):
    stop = stopwords.words("english")
    return [word for word in text if word not in stop]

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

def lem(words):
    lem_list = []
    lemmatizer = WordNetLemmatizer()
    for word in words:
        lem_list.append(lemmatizer.lemmatize(word))
    return lem_list

def clean_text(text):
#     text = str.lower(text) #lower case
    text = re.sub(r"http\S+", "", text) #rm websites
    text = re.sub(r"\s+[a-zA-Z]\s+", " ", text) # rm single characters 
    text = re.sub(r"\s\s+","", text) # rm multiple spaces
    text = re.sub(r"#","", text) # rm hashtags
    text = re.sub(r"[(,?!)]","", text) # rm special characters 
    text = re.sub("@[^\s]+", "", text) # rm @twitter_id
#     text = remove_emoji(text) # rm emoji
    #text = re.sub("<[^>]*>", "", text) # rm HTML markup
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = (re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")).strip()
    
   
    return text

tweets["text"] = tweets["text"].apply(lambda x: clean_text(x))
tweets["stemmed"] = tweets["text"].apply(lambda x: tokenizer_porter(x))
tweets["stop_removed"] = tweets["stemmed"].apply(lambda x: remove_stop_words(x))
tweets["lemmatized"] = tweets["stop_removed"].apply(lambda x: lem(x))

tweets.head()

## Vectorize
### CountVectorizer contstructs a bag-of-words model 

In [4]:
count = CountVectorizer()
docs = np.array(tweets["text"])
bag = count.fit_transform(docs)

## Td-idf

In [6]:
tfidf = TfidfVectorizer(preprocessor="".join,
                         use_idf=True, 
                         norm = "l2",
                         smooth_idf=True)
np.set_printoptions(precision=2)
tfidf_vectors = tfidf.fit_transform(docs)